In [1]:
import numpy as np
import itertools # para generar combinarion de elemntos de una lista

CONCEPTOS PARA EL ALGORITMO APRIORI

SUPPORT = cantidad item A / NRO de itemsets

CONFIDENCE(A -> B)
CONFIDENCE = cantidad de A y B / cantidad item A 

LIFT(A -> B)
LIFT = confidence(A -> B) / suport(B)

In [99]:
data = np.load('/Users/blablabla/usuario_alojado/SEMESTRE_II_2021/Mineria-de-datos/Actividad03/spotify.npy', allow_pickle=True)
data1 = data.item(0)

dataTest = {0:['Pan','Leche','Pañales'],
           1:['Pan','Pañales','Cerveza','Huevos'],
           2:['Leche','Pañales','Cerveza','Refresco','Café'],
           3:['Pan','Leche','Pañales','Cerveza'],
           4:['Pan','Refresco','Leche','Pañales']}

In [100]:
def candidateGet(itemSetsFrecuentes, length): #retorna una lista con los nuevos superItemSets 
    superItemSets = []
    for index, itemSet in enumerate(itemSetsFrecuentes):
        for j in range(index+1,len(itemSetsFrecuentes)):
            union = np.append(itemSet,itemSetsFrecuentes[j])
            union = list(set(union))
            union = sorted(union)
            #print(union)
            if len(union) == length and union not in superItemSets:
                superItemSets.append(union)
    return superItemSets

In [101]:
def candidatePrune( newItemSets, itemSetsFrecuentes): # retorna un arreglo
    pruneList = []
    for itemSet in newItemSets:
        cont = 0 # para validar que todos sus subItemSet sean frecuentes
        combinaciones = list(itertools.combinations(itemSet,len(itemSet)-1))
        #print(len(combinaciones), combinaciones)
        for element in combinaciones:
            #print(element, list(element))
            if list(element) in itemSetsFrecuentes:
                cont += 1
        #print(cont)
        if cont == len(combinaciones):
            pruneList.append(itemSet)
    return pruneList

In [102]:
def subset(Candidatos, itemSetOriginal): # retorna una lista de los candidatos que se encuentran en el ItemSetOriginal
    encontrados = []
    for candidato in Candidatos:
        contador = 0 # para validar que se encontraron todos los items dentro del itemSetOriginal
        for item in candidato:
            if (item in itemSetOriginal):
                contador += 1
        if contador == len(candidato):
            encontrados.append(candidato)
    return encontrados

In [103]:
def get_frequent_itemsets(playlists, min_support): # min_supoort en el rango de [0,1]
    # VARIABLES 
    nroItemSets = len(playlists) # SUPPORT = cantidadItem / nroItemSets
    itemSetsFrecuentes = [] # lista que almacena los itemsets frecuentes 
    listSupport = [] # Lista que almacena los suports de los itemsets frecuentes
    
    # Encontrar los 1-itemsets frecuentes
    frecuencias = {} # acumula las item junsto a su nro de veces itemsets en los que aparece
    musicas = [] # guarda los items individuales
    noRepetidos = [] # es un filtro de la anterior lista, lista sin repetidos
    frecuentes = [] # 
    for i in playlists:
        for j in playlists[i]:
            musicas.append(j)
    noRepetidos = sorted(set(musicas))
    #print(noRepetidos)
    for i in noRepetidos:
        frecuencias[i] = musicas.count(i)
    for i in frecuencias:
        if frecuencias[i]/nroItemSets >= min_support:
            frecuentes.append([i])
            itemSetsFrecuentes.append([i])
            listSupport.append(frecuencias[i]/nroItemSets)
    # Bucle para encontrar n-itemsets frecuentes mientras existan.
    #VARIABLES
    #print(frecuentes)
    k = 1
    FK = [] # lista de itemsets frecuentes de K-items 
    while (len(frecuentes) > 1):
        k += 1
        FK = candidateGet(frecuentes,k) # generar K-itemsets candidatos 
        FK = candidatePrune(FK, frecuentes) # podar candidatos que tengan sub-itemsets no frecuentes
        SUPPORT = [0]*len(FK) # creamos un arreglo que va a almacenar los support
        #print(playlists)
        for playlist in playlists: # bucle para calcular su SUPPORT y definir los nuevos K-itemsets frecuentes
            #print(playlist)
            encontrados = subset(FK, playlists[playlist])  # revisar que supergrupos de FK estan en la playlist
            for encontrado in encontrados:
                SUPPORT[FK.index(encontrado)] += 1
        frecuentes = []
        for index,candidato in enumerate(FK):
            #print(candidato)
            #print(SUPPORT[index]/nroItemSets)
            if (SUPPORT[index]/nroItemSets >= min_support):
                itemSetsFrecuentes.append(candidato)
                listSupport.append(SUPPORT[index]/nroItemSets)
                frecuentes.append(candidato)
    return itemSetsFrecuentes, listSupport
    
        

In [104]:
frecuentes, Supports = get_frequent_itemsets(dataTest,0.5)
print(frecuentes)
print(Supports)

[['Cerveza'], ['Leche'], ['Pan'], ['Pañales'], ['Cerveza', 'Pañales'], ['Leche', 'Pan'], ['Leche', 'Pañales'], ['Pan', 'Pañales'], ['Leche', 'Pan', 'Pañales']]
[0.6, 0.8, 0.8, 1.0, 0.6, 0.6, 0.8, 0.8, 0.6]


In [105]:
def rulesCandidateGen( rules, lengt):
    superItemSets = []
    for index, itemSet in enumerate(rules):
        for j in range(index,len(rules)):
            union = np.append(itemSet,rules[j])
            union = set(union)
            union = sorted(union)
            if len(union) == lengt and union not in superItemSets:
                superItemSets.append(union)
    return superItemSets

In [106]:
def rulesCandidatePrune( candidatos, subCandidatos):
    candidatePrune = []
    for candidato in candidatos:
        cont = 0
        if (len(candidato)-1 == 0):
            subCombinaciones = candidatos
        else:
            subCombinaciones = list(itertools.combinations(candidato, len(candidato)-1))
        for item in subCombinaciones:
            if list(item) in subCandidatos:
                cont += 1
        if cont == len(subCombinaciones):
            candidatePrune.append(candidato)
    return candidatePrune

In [115]:
def generate_association_rules(frequent_itemsets, supports, confidence = 0, lift = 0):
    # VARIABLES
    rules = [] # almacena las reglas que superen el confidence y ordenadas por el lift
    _confi = [] # almacena el confidence de las reglas
    _lift = [] # almacena el lift de las reglas
    listPoda = [] # almacena los subconjuntos que no superen el confidence
    
    #
    for index, itemset in enumerate(frequent_itemsets):
        # convertimos los elementos de itemset en listas
        newItemset = [[i] for i in itemset]
        k = len(newItemset) # tamaño del itemset
        m = 1 # tamaño inicial del consecuente de la regla
        H = newItemset # inicializacion de la lista que almacenara las reglas generadas de itemset 
        #print(k, itemset)
        while k > m:
            candidatos = rulesCandidateGen(H, m) # generamos nuevas reglas 
            candidatos = rulesCandidatePrune( candidatos, H) # podamos algunas reglas
            H = []
            for candidato in candidatos:
                _support = frequent_itemsets.index(candidato) # calculamos el indice del support para el candidato
                conf = supports[index] / supports[_support]
                if conf >= confidence:
                    # generar la regla que tiene la forma de [[antecedente],[precedente]]
                    antecedente = candidato
                    precedente  = list(sorted(set(itemset).difference(set(candidato))))
                    #print("precedente: ",precedente)
                    rules.append([antecedente,precedente])
                    _confi.append(conf)
                    # Calcular el lift
                    lift = conf / supports[frequent_itemsets.index(precedente)]
                    _lift.append(lift)
                    H.append(candidato)
            m += 1
    
    return rules, _lift
        

In [117]:
reglas, listLift = generate_association_rules(frecuentes, Supports, confidence = 0, lift = 0)
for index, regla in enumerate(reglas):
    print(regla, listLift[index])

[['Cerveza'], ['Pañales']] 1.0
[['Pañales'], ['Cerveza']] 1.0
[['Leche'], ['Pan']] 0.9374999999999998
[['Pan'], ['Leche']] 0.9374999999999998
[['Leche'], ['Pañales']] 1.0
[['Pañales'], ['Leche']] 1.0
[['Pan'], ['Pañales']] 1.0
[['Pañales'], ['Pan']] 1.0
[['Leche'], ['Pan', 'Pañales']] 0.9374999999999998
[['Pan'], ['Leche', 'Pañales']] 0.9374999999999998
[['Pañales'], ['Leche', 'Pan']] 1.0
[['Leche', 'Pan'], ['Pañales']] 1.0
[['Leche', 'Pañales'], ['Pan']] 0.9374999999999998
[['Pan', 'Pañales'], ['Leche']] 0.9374999999999998
